### You are given a certain number of employees and certain number of calls / customer interval , and average time to support the customer ,given a simulation time of 2 hrs how many customer we are going to handle

In [22]:
import random
import simpy
import numpy as np


In [23]:
NUM_EMPLOYEES = 3#addding number of employees doesnt solvve the problems
# all times are in mins
AVG_SUPPORT_TIME = 5
CUSTOMER_INTERVAL = 2
SIM_TIME = 120
customers_handled = 0


In [24]:
class CallCenter:
    def __init__(self,env,num_employees, support_time):
        self.env = env
        self.staff = simpy.Resource(env, num_employees)
        self.support_time = support_time
        
    def support(self,customer):
        random_time = max(1,np.random.normal(self.support_time,4))
        yield self.env.timeout(random_time)
        print(f'Support finished for {customer} at {self.env.now:.2f}')
        

def customer(env,name,call_center):
    global customers_handled
    print(f"Customer  {name} enters the waiting Queue {env.now:.2f}")
    with call_center.staff.request() as request:
        yield request
        print(f"Customer {name} enters call at {env.now:.2f}")
        yield env.process(call_center.support(name))
        print(f"Customer {name} left call at {env.now:.2f}")
        customers_handled += 1
        

        
            
        

In [25]:
def setup(env, num_employees, support_time, customer_interval):
    call_center = CallCenter(env, num_employees, support_time)
    for i  in range(1,6):
        env.process(customer(env,i, call_center))
    
    while True:
        yield env.timeout(random.randint(customer_interval - 1,customer_interval + 1))
        i += 1
        env.process(customer(env,num_employees, call_center))
        

In [26]:
print("Starting call center simulation")
env = simpy.Environment()
env.process(setup(env, NUM_EMPLOYEES, AVG_SUPPORT_TIME, CUSTOMER_INTERVAL))
env.run(until=SIM_TIME)
print("Customers handled: " + str(customers_handled))

Starting call center simulation
Customer  1 enters the waiting Queue 0.00
Customer  2 enters the waiting Queue 0.00
Customer  3 enters the waiting Queue 0.00
Customer  4 enters the waiting Queue 0.00
Customer  5 enters the waiting Queue 0.00
Customer 1 enters call at 0.00
Customer 2 enters call at 0.00
Customer 3 enters call at 0.00
Customer 4 enters call at 0.00
Customer 5 enters call at 0.00
Customer  9 enters the waiting Queue 1.00
Support finished for 4 at 1.00
Customer 9 enters call at 1.00
Customer 4 left call at 1.00
Support finished for 1 at 1.61
Customer 1 left call at 1.61
Support finished for 3 at 1.62
Customer 3 left call at 1.62
Support finished for 2 at 1.81
Customer 2 left call at 1.81
Support finished for 5 at 2.12
Customer 5 left call at 2.12
Customer  9 enters the waiting Queue 3.00
Customer 9 enters call at 3.00
Support finished for 9 at 5.46
Customer 9 left call at 5.46
Customer  9 enters the waiting Queue 6.00
Customer 9 enters call at 6.00
Customer  9 enters the w

In [27]:
print("Customers handled: " + str(customers_handled))

Customers handled: 57
